In [1]:
import torch
from torch import optim
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
# from torchvision.transforms import ToTensorfrom 
from torch.utils.tensorboard import SummaryWriter

import matplotlib.pyplot as plt
import numpy as np
import os

In [2]:
class PointCloudDataset(Dataset):
    def __init__(self, pts_file):
        points = np.loadtxt(pts_file, delimiter=' ')
        points = np.delete(points, -2, 1)
        points = np.delete(points, -2, 1)
        points = np.delete(points, -2, 1)

        print(points.shape)
        # do i need to min max intensity, return number, number of returns, etc.? probably not
        for i in range(3):
            dim_min, dim_max = min(points[:,i]), max(points[:,i])
            points[:,i] = (points[:,i] - dim_min) / (dim_max - dim_min)
        self.data = points
        
    def __len__(self):
        return len(self.data) // 25000
        
    def __getitem__(self, idx):
        # Return batches of 2500 points
        xyzirn = self.data[idx * 25000: (idx + 1) * 25000, :3]  # x, y, z, ***intensity, return number, number of returns***
        label = self.data[idx * 25000: (idx + 1) * 25000, 3] == 8

        xyzirn = torch.from_numpy(xyzirn.T).float()
        label = torch.tensor(label).long()
        
        return xyzirn, label

training_data = PointCloudDataset(r"C:\Users\and13375\Documents\3D Point Segmentation\Test\Vaihingen\3DLabeling\Vaihingen3D_Traininig.pts")
testing_data = PointCloudDataset(r"C:\Users\and13375\Documents\3D Point Segmentation\Test\Vaihingen\3DLabeling\Vaihingen3D_EVAL_WITH_REF.pts")
train_dataloader = DataLoader(training_data, batch_size=10, shuffle=False) # can/should i use shuffle, try lowering it?
test_dataloader = DataLoader(testing_data, batch_size=10, shuffle=False)

num_classes = 2

(753876, 4)
(411722, 4)


In [3]:
train_features, train_labels = next(iter(train_dataloader))
print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")

Feature batch shape: torch.Size([10, 3, 25000])
Labels batch shape: torch.Size([10, 25000])


In [4]:
import open3d as o3d
import numpy as np
from collections import Counter

def plot_batch(entire_view = True):
    # Assuming pc is your point cloud data, in shape (N, 3)
    pc_num = 0

    if entire_view:
        pc = training_data.data[:, :3]
        labels = training_data.data[:, 3] == 8
    else:
        pc = training_data[pc_num][0].T.view(-1,3).numpy()
        labels = training_data[pc_num][1].numpy()

    print(Counter(labels), len(labels))

    # Define colors for each label
    color_map = {0: [0.5, 0.5, 0.5],  # Gray color for label 0
                1: [1.0, 0.0, 0.0]}  # Red color for label 1

    # Map each label to a color
    colors = np.array([color_map[label] for label in labels])

    # Create point cloud
    point_cloud = o3d.geometry.PointCloud()
    point_cloud.points = o3d.utility.Vector3dVector(pc)
    point_cloud.colors = o3d.utility.Vector3dVector(colors)

    # Visualize the point cloud
    o3d.visualization.draw_geometries([point_cloud])
# plot_batch(entire_view = False)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [5]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
# device = 'cpu'
print(f"Using {device} device")

Using cuda device


In [6]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.utils.data
from torch.autograd import Variable
import numpy as np
import torch.nn.functional as F


class STN3d(nn.Module):
    def __init__(self):
        super(STN3d, self).__init__()
        self.conv1 = torch.nn.Conv1d(3, 64, 1)
        self.conv2 = torch.nn.Conv1d(64, 128, 1)
        self.conv3 = torch.nn.Conv1d(128, 1024, 1)
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 9)
        self.relu = nn.ReLU()

        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(1024)
        self.bn4 = nn.BatchNorm1d(512)
        self.bn5 = nn.BatchNorm1d(256)


    def forward(self, x):
        batchsize = x.size()[0]
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        x = torch.max(x, 2, keepdim=True)[0]
        x = x.view(-1, 1024)

        x = F.relu(self.bn4(self.fc1(x)))
        x = F.relu(self.bn5(self.fc2(x)))
        x = self.fc3(x)

        iden = Variable(torch.from_numpy(np.array([1,0,0,0,1,0,0,0,1]).astype(np.float32))).view(1,9).repeat(batchsize,1)
        if x.is_cuda:
            iden = iden.cuda()
        x = x + iden
        x = x.view(-1, 3, 3)
        return x


class STNkd(nn.Module):
    def __init__(self, k=64):
        super(STNkd, self).__init__()
        self.conv1 = torch.nn.Conv1d(k, 64, 1)
        self.conv2 = torch.nn.Conv1d(64, 128, 1)
        self.conv3 = torch.nn.Conv1d(128, 1024, 1)
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, k*k)
        self.relu = nn.ReLU()

        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(1024)
        self.bn4 = nn.BatchNorm1d(512)
        self.bn5 = nn.BatchNorm1d(256)

        self.k = k

    def forward(self, x):
        batchsize = x.size()[0]
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        x = torch.max(x, 2, keepdim=True)[0]
        x = x.view(-1, 1024)

        x = F.relu(self.bn4(self.fc1(x)))
        x = F.relu(self.bn5(self.fc2(x)))
        x = self.fc3(x)

        iden = Variable(torch.from_numpy(np.eye(self.k).flatten().astype(np.float32))).view(1,self.k*self.k).repeat(batchsize,1)
        if x.is_cuda:
            iden = iden.cuda()
        x = x + iden
        x = x.view(-1, self.k, self.k)
        return x

class PointNetfeat(nn.Module):
    def __init__(self, global_feat = True, feature_transform = True):
        super(PointNetfeat, self).__init__()
        self.stn = STN3d()
        self.conv1 = torch.nn.Conv1d(3, 64, 1)
        self.conv2 = torch.nn.Conv1d(64, 128, 1)
        self.conv3 = torch.nn.Conv1d(128, 1024, 1)
        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(1024)
        self.global_feat = global_feat
        self.feature_transform = feature_transform
        if self.feature_transform:
            self.fstn = STNkd(k=64)

    def forward(self, x):
        n_pts = x.size()[2]
        trans = self.stn(x)
        x = x.transpose(2, 1)
        x = torch.bmm(x, trans)
        x = x.transpose(2, 1)
        x = F.relu(self.bn1(self.conv1(x)))

        if self.feature_transform:
            trans_feat = self.fstn(x)
            x = x.transpose(2,1)
            x = torch.bmm(x, trans_feat)
            x = x.transpose(2,1)
        else:
            trans_feat = None

        pointfeat = x
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.bn3(self.conv3(x))
        x = torch.max(x, 2, keepdim=True)[0]
        x = x.view(-1, 1024)
        if self.global_feat:
            return x, trans, trans_feat
        else:
            x = x.view(-1, 1024, 1).repeat(1, 1, n_pts)
            return torch.cat([x, pointfeat], 1), trans, trans_feat


class PointNetDenseCls(nn.Module):
    def __init__(self, k = 2, feature_transform=True):
        super(PointNetDenseCls, self).__init__()
        self.k = k
        self.feature_transform=feature_transform
        self.feat = PointNetfeat(global_feat=False, feature_transform=feature_transform)
        self.conv1 = torch.nn.Conv1d(1088, 512, 1)
        self.conv2 = torch.nn.Conv1d(512, 256, 1)
        self.conv3 = torch.nn.Conv1d(256, 128, 1)
        self.conv4 = torch.nn.Conv1d(128, self.k, 1)
        self.bn1 = nn.BatchNorm1d(512)
        self.bn2 = nn.BatchNorm1d(256)
        self.bn3 = nn.BatchNorm1d(128)

    def forward(self, x):
        batchsize = x.size()[0]
        n_pts = x.size()[2]
        x, trans, trans_feat = self.feat(x)
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        x = self.conv4(x)
        x = x.transpose(2,1).contiguous()
        x = F.log_softmax(x.view(-1,self.k), dim=-1)
        x = x.view(batchsize, n_pts, self.k)
        return x, trans, trans_feat
    
# model = PointNetDenseCls(k=2).to(device)

In [7]:
# for name, module in model.named_modules():
#     if name == 'mlp8.1':
#         print(name)
#     # print(name)

In [8]:
def feature_transform_regularizer(trans):
    d = trans.size(1)
    I = torch.eye(d).unsqueeze(0).to(device)
    loss = torch.linalg.norm(I - torch.bmm(trans, trans.transpose(2,1)), dim=(1,2))
    loss = torch.mean(loss)
    return loss

In [9]:
class CustomLoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.cross_entropy_loss = nn.CrossEntropyLoss()

    def forward(self, inputs, targets, trans):
        d = trans.size(1)
        I = torch.eye(d).unsqueeze(0).to(device)
        loss = torch.linalg.norm(I - torch.bmm(trans, trans.transpose(2,1)), dim=(1,2))
        loss = torch.mean(loss)
        return self.cross_entropy_loss(inputs, targets) + loss

## Online Code

In [10]:
from sklearn.metrics import f1_score
from collections import Counter


writer = SummaryWriter()
loss_func = CustomLoss()
classifier = PointNetDenseCls()

optimizer = optim.Adam(classifier.parameters(), lr=0.001, betas=(0.9, 0.999))
# scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5)
classifier.to(device)

num_batch = len(training_data)

for epoch in range(80):
    # scheduler.step()
    for i, data in enumerate(train_dataloader, 1):
        points, target = data
        # points = points.transpose(2, 1)
        points, target = points.to(device), target.to(device)
        optimizer.zero_grad()
        classifier = classifier.train()
        pred, _, trans_feat = classifier(points)
        pred = pred.view(-1, num_classes)
        target = target.view(-1, 1).squeeze()
        print(pred.size(), target.size())

        # loss = loss_func(pred, target, trans_feat)

        loss = F.nll_loss(pred, target)
        loss += feature_transform_regularizer(trans_feat) * 0.001
        loss.backward()

        optimizer.step()
        pred_choice = pred.data.max(1)[1]
        correct = pred_choice.eq(target.data).cpu().sum()
        f1 = f1_score(pred_choice.cpu(), target.cpu())

        writer.add_scalar('training loss', loss.item(), global_step=epoch * len(train_dataloader) + i-1)
        writer.add_scalar('training f1 score', f1, global_step=epoch * len(train_dataloader) + i-1)

        print('[%d: %d/%d] train loss: %f accuracy: %f f1 score: %f' % (epoch+1, i, 3, loss.item(), correct.item()/float(10 * 25000), f1))


writer.flush()
writer.close()

torch.Size([250000, 2]) torch.Size([250000])
[1: 2/3] train loss: 0.772034 accuracy: 0.668476 f1 score: 0.268398
torch.Size([250000, 2]) torch.Size([250000])
[1: 3/3] train loss: 0.711069 accuracy: 0.657088 f1 score: 0.155089
torch.Size([250000, 2]) torch.Size([250000])
[1: 4/3] train loss: 0.696830 accuracy: 0.783752 f1 score: 0.182464
torch.Size([250000, 2]) torch.Size([250000])
[2: 2/3] train loss: 0.772995 accuracy: 0.648488 f1 score: 0.171463
torch.Size([250000, 2]) torch.Size([250000])
[2: 3/3] train loss: 0.630760 accuracy: 0.889620 f1 score: 0.002025
torch.Size([250000, 2]) torch.Size([250000])
[2: 4/3] train loss: 0.656365 accuracy: 0.769084 f1 score: 0.129630
torch.Size([250000, 2]) torch.Size([250000])


In [ ]:
from sklearn.metrics import f1_score

outputs = None
points = None
x = y = None
def test_PointNet(pointnet, test_dataloader, device):
    global outputs, points, x, y
    pointnet.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for idx, (points, labels) in enumerate(test_dataloader):
            points, labels = points.to(device), labels.to(device)
            outputs, _, _ = pointnet(points)
            _, predicted = torch.max(outputs.data, 2)
            total += labels.size(0) * labels.size(1)
            correct += (predicted == labels).sum().item()
            x = labels.view(-1).to('cpu').numpy()
            y = predicted.view(-1).to('cpu').numpy()
            f1 = f1_score(x, y)
            print('F1 score: ', f1)
    
test_PointNet(classifier, test_dataloader, device)

F1 score:  0.12266352592204847
F1 score:  0.0018165304268846505


In [ ]:
print(sum(x))
print(sum(y))
print(outputs)

41128
0
tensor([[[-0.5913, -0.8065],
         [-0.5913, -0.8065],
         [-0.5914, -0.8065],
         ...,
         [-0.5921, -0.8056],
         [-0.5921, -0.8056],
         [-0.5920, -0.8056]],

        [[-0.5918, -0.8059],
         [-0.5924, -0.8053],
         [-0.5923, -0.8053],
         ...,
         [-0.5911, -0.8068],
         [-0.5911, -0.8068],
         [-0.5911, -0.8068]],

        [[-0.5921, -0.8056],
         [-0.5921, -0.8056],
         [-0.5921, -0.8056],
         ...,
         [-0.5929, -0.8046],
         [-0.5929, -0.8046],
         [-0.5929, -0.8046]],

        ...,

        [[-0.5876, -0.8112],
         [-0.5876, -0.8112],
         [-0.5876, -0.8112],
         ...,
         [-0.5872, -0.8116],
         [-0.5872, -0.8116],
         [-0.5872, -0.8116]],

        [[-0.5877, -0.8110],
         [-0.5877, -0.8110],
         [-0.5877, -0.8110],
         ...,
         [-0.5878, -0.8110],
         [-0.5880, -0.8107],
         [-0.5878, -0.8110]],

        [[-0.5888, -0.8097],

In [ ]:
print(f"Labels has {sum(x == 0)} zeroes and {sum(x == 1)} ones")
print(f"Predictions has {sum(y == 0)} zeroes and {sum(y == 1)} ones")


# writer.add_graph(model, points)
# writer.flush()
# writer.close()

Labels has 208872 zeroes and 41128 ones
Predictions has 250000 zeroes and 0 ones


73.4029268292683% on roofs (5)
86.77414634146342% on trees (8)